In [81]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import sys
import numpy as np

In [82]:
with open('api-key.txt') as f:
    key = f.read()
client = OpenAI(api_key=key)

In [83]:
documents = []
s = ""
with open('hadestwo.txt') as f:
    for line in f:
        s += line
documents.append(s)

s = ""
with open('mcpvp.txt') as f:
    for line in f:
        s += line
documents.append(s)

s = ""
with open('fantasie.txt', errors = 'ignore') as f:
    for line in f:
        s += line
documents.append(s)

print(documents)


['Hades II is an upcoming roguelike action role-playing game video game developed and published by Supergiant Games, serving as a sequel to Hades (2020). It was announced in December 2022 and was released in early access in May 2024 for Windows and in October 2024 for macOS, with plans to bring the game to consoles after the early access period. The game follows MelinoÃ«, Princess of the Underworld and sister to Zagreus, the protagonist of the first game. MelinoÃ« aims to defeat Chronos, the Titan of Time, with the aid of other Olympian Gods. \n\nLike the first game, Hades II is a roguelike dungeon crawler game. Set "sometime" after the events of Hades in the same universe, the sequel stars MelinoÃ«, Princess of the Underworld and sister to the protagonist of the first game, Zagreus. Alongside her mentor Hecate, she aims to defeat Chronos, the titan of time, who returned to enact revenge on all gods and mortals who opposed him in the past and took her family prisoner. Returning from Ha

In [84]:
# Function to create embeddings
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

try:
    document_embeddings = np.load('document_embeddings.npy', allow_pickle=True)
except FileNotFoundError:
    document_embeddings = [get_embedding(doc) for doc in documents]
    np.save('document_embeddings.npy', document_embeddings)

In [85]:
import chromadb
from chromadb.utils import embedding_functions

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient()

# Create or get a collection in ChromaDB
collection_name = "basic_collection"

collection = chroma_client.get_or_create_collection(name = collection_name)


# Add documents and their embeddings to the collection
for index, (doc, embedding) in enumerate(zip(documents, document_embeddings)):
    collection.add(
        ids=[f"doc_{index}"],
        documents=[doc],
        embeddings=[embedding]
    )


Add of existing embedding ID: doc_0
Insert of existing embedding ID: doc_0
Add of existing embedding ID: doc_1
Insert of existing embedding ID: doc_1
Add of existing embedding ID: doc_2
Insert of existing embedding ID: doc_2


In [ ]:
query = input("Enter your query: ")

query_embedding = get_embedding(query)

k = 1

# Search for similar documents
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=k
)

retrieved_docs = results['documents'][0]

total_context = "" 
for doc in retrieved_docs:
    total_context += doc
    total_context += "\n"


["We are going to talk about Chopin's Fantasie Impromptu today. \nThis Fantaisie-Impromptu was written in 1834, but Chopin didnâ€™t publish it in his lifetime. In fact, he specifically requested that none of his unpublished works be published after his death â€“ though this was completely ignored and there are quite a few posthumous compositions of his that we now enjoy.\nThis piece was written in the key of C# minor, just like Beethovenâ€™s famous Moonlight Sonata, and that isnâ€™t the only similarity between the two. More on that in a moment.\nFantaisie-Impromptu is one of Chopinâ€™s most famous and most performed compositions, and itâ€™s been all over popular media â€“ youâ€™ve likely heard it at least once in your life.\nThis composition is the fourth in a series of four Impromptus written by Chopin â€“ the other three were published in his lifetime.\nSo why did Chopin leave this one unpublished?\nWell, Chopin dedicated it to his close friend Julian Fontana, and Julian was the one 

In [ ]:


system_message = """ you are an expert in the fields of music and video games. You have especially studied the content and development of hades 2. You are additionally a minecraft pvp expert. Finally, you are a chopin specialist with great knowledge of his works.
"""
user_message = f""" 
Based on the following context, provide a detailed answer to this question

Context:
{total_context}

Question:
{query}

Answer:
"""


# Print the entire prompt
print("\nPrompt:")
print(system_message)
print(user_message)

# Generate a response using the language model
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [{"role" : "system", "content": system_message},
    {"role" : "user", "content" : user_message}
    ],
    max_tokens=300,
    temperature=0.7
)

# Extract and print the answer
answer = response.choices[0].message.content
print("\nAnswer:")
print(answer)


Prompt:

You are an expert in the fields of video game history, especially indie games such as hades 2, the music of Chopin, and minecraft 1.8 player vs player combat

Based on the following context, provide a detailed answer to the question.

Context:
We are going to talk about Chopin's Fantasie Impromptu today. 
This Fantaisie-Impromptu was written in 1834, but Chopin didnâ€™t publish it in his lifetime. In fact, he specifically requested that none of his unpublished works be published after his death â€“ though this was completely ignored and there are quite a few posthumous compositions of his that we now enjoy.
This piece was written in the key of C# minor, just like Beethovenâ€™s famous Moonlight Sonata, and that isnâ€™t the only similarity between the two. More on that in a moment.
Fantaisie-Impromptu is one of Chopinâ€™s most famous and most performed compositions, and itâ€™s been all over popular media â€“ youâ€™ve likely heard it at least once in your life.
This composition 